In [1]:
!pip -q install langchain huggingface_hub transformers llama_hub llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.9/103.9 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
%pip install llama-index-llms-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [3]:
from typing import Any, Callable, Dict
from llama_index.llms.huggingface import HuggingFaceInferenceAPI
from huggingface_hub import AsyncInferenceClient, InferenceClient
from llama_index.core.base.llms.types import (
    CompletionResponseGen,
    CompletionResponse
)

class CustomLLMInferenceWrapper(HuggingFaceInferenceAPI):

    def __init__(self, **kwargs: Any):
           super().__init__(**kwargs)
           model_name=kwargs.get("model_name")
           self._sync_client = InferenceClient(model=model_name)



    def stream_complete(
        self, prompt: str, formatted: bool = False, **kwargs: Any
        ) -> CompletionResponseGen:
        """Streaming completion endpoint."""

        # Create a generator based on the streamer
        def gen() -> CompletionResponseGen:
            for response in self._sync_client.text_generation(prompt,**kwargs, stream=True, details=True, return_full_text=False):
                yield CompletionResponse(text=response.token.text,delta=response.token.text)
        return gen()

In [14]:
kwargs = dict(
        temperature=0.2,
        max_new_tokens=512,
        top_p=0.95,
        repetition_penalty=0.93,
        do_sample=True,
        seed=42,
    )
def format_prompt(message):
  prompt = f"[INST] {message} [/INST]"
  return prompt

llm = CustomLLMInferenceWrapper(model_name="mistralai/Mixtral-8x7B-Instruct-v0.1")

In [19]:
query = "can i know about Bangladesh ?"
prompt = format_prompt(query)
response = llm.stream_complete(prompt,**kwargs)

for result in response:
    print(result.text,end='')

 Sure! I'd be happy to share some information about Bangladesh.

Bangladesh is a country located in South Asia, bordering India and Myanmar. It is situated on the Bay of Bengal and has a population of over 160 million people, making it one of the most densely populated countries in the world.

The capital and largest city of Bangladesh is Dhaka, a bustling metropolis that is known for its rich history, vibrant culture, and delicious cuisine. Other major cities in Bangladesh include Chittagong, Sylhet, and Rajshahi.

Bangladesh has a diverse and rich culture, with influences from Hinduism, Buddhism, Islam, and Christianity. The official language of Bangladesh is Bengali, which is spoken by the majority of the population.

The country has a tropical monsoon climate, with a hot and humid summer and a cooler winter. The monsoon season brings heavy rainfall, which can sometimes lead to flooding.

Bangladesh is known for its beautiful landscapes, including the Sundarbans, the world's largest